# Main Runner

In [1]:
import pandas as pd

from objects import *
import pickle
import os

In [2]:
# Get all files in the experiments folder
experiments = os.listdir('experiments')
# read in pickle object
with open(f"experiments/{experiments[0]}", 'rb') as f:
    exp = pickle.load(f)

In [3]:
csmpo = DayTradingPolicy(exp)

In [4]:
market_sim = MarketSimulator(exp,csmpo)

In [6]:
exp.truth

component,NOC_24766,EBAY_86356,TFCFA_90441,MON_88668,HBAN_42906,DIS_26403,LMT_21178,BALL_57568,NTAP_82598,AGN_75646,...,TGNA_47941,POM_23501,ZION_84129,MTB_35554,WY_39917,YHOO_83435,HST_46703,CELG_11552,BCR_46877,AES_76712
date,,,,,,,,,,,,,,,,,,,,,
2014-11-04,138.57,53.270,33.330,113.39,9.905,90.35,190.79,63.93,43.020,195.12,...,30.66,27.39,29.380,121.27,34.46,47.080,22.96,105.75,163.62,13.76
2014-11-05,137.84,53.585,34.840,113.82,10.040,91.00,189.73,64.50,43.200,196.32,...,30.59,27.50,29.500,122.83,34.38,47.460,22.88,105.15,164.58,13.95
2014-11-06,138.11,53.260,35.310,114.51,10.080,92.00,189.88,64.69,43.170,197.40,...,31.02,27.34,29.690,124.43,34.24,47.930,22.79,107.14,164.97,13.06
2014-11-07,136.11,53.515,34.610,114.44,10.090,90.00,186.15,64.97,42.300,196.00,...,30.31,27.46,29.670,124.54,34.30,48.550,22.66,106.45,162.38,13.46
2014-11-10,135.65,53.890,34.740,116.89,10.130,89.78,186.99,65.90,42.150,195.24,...,30.76,27.59,29.670,124.66,34.41,49.410,22.76,108.01,163.87,13.63
2014-11-11,136.85,54.190,34.930,118.31,10.120,89.98,187.12,65.89,42.270,195.34,...,30.83,27.51,29.590,124.85,34.25,49.050,22.76,107.85,164.27,13.49
2014-11-12,136.02,54.060,34.640,118.99,10.170,89.90,186.87,65.55,42.310,196.51,...,30.87,27.55,29.760,125.62,34.03,50.600,22.68,108.25,165.21,13.44
2014-11-13,137.20,54.370,34.990,119.47,10.095,90.49,187.97,65.16,41.085,196.21,...,30.88,27.30,29.280,125.36,33.95,50.500,22.73,107.43,165.59,13.69
2014-11-14,136.07,54.360,35.110,118.97,10.100,90.80,186.60,64.24,42.270,198.65,...,31.05,27.28,29.140,124.94,33.90,51.750,22.67,104.05,164.04,13.47


In [5]:
final_portf = market_sim.run()

Starting Simulation


  0%|          | 0/30 [00:00<?, ?it/s]

Current Portfolio Value at 2014-11-03: 259697.0
	 Optimizing with 31 time steps, 2005 constraints, and 2604 variables...
	 Optimized. Time taken:  37.5718834400177


  3%|▎         | 1/30 [00:37<18:21, 37.98s/it]

Current Portfolio Value at 2014-11-04: 252966.37999999998
	 Optimizing with 30 time steps, 1941 constraints, and 2520 variables...
	 Optimized. Time taken:  36.50354790687561


...

 93%|█████████▎| 28/30 [02:30<00:00,  2.79it/s]

Current Portfolio Value at 2014-12-11: 267533.11999999994
	 Optimizing with 3 time steps, 213 constraints, and 252 variables...
	 Optimized. Time taken:  0.0


 97%|█████████▋| 29/30 [02:30<00:00,  3.06it/s]

Current Portfolio Value at 2014-12-12: 264373.8799999999
	 Optimizing with 2 time steps, 149 constraints, and 168 variables...
	 Optimized. Time taken:  0.0


100%|██████████| 30/30 [02:30<00:00,  5.03s/it]
